In [34]:
!pip install kaggle

In [36]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [37]:
!kaggle competitions download -c fake-news

 88% 41.0M/46.5M [00:00<00:00, 95.1MB/s]
100% 46.5M/46.5M [00:00<00:00, 94.0MB/s]


In [40]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [41]:
!unzip train.csv.zip
!unzip test.csv.zip

unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.
unzip:  cannot find or open test.csv.zip, test.csv.zip.zip or test.csv.zip.ZIP.


** importing data**

In [44]:
import pandas as pd
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


**manipulating** **data**

In [45]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [48]:
data = df.dropna()
data.reset_index(inplace = True)

In [49]:
x = data.drop("label",axis = 1)
y = data.label
x,y

(       index  ...                                               text
 0          0  ...  House Dem Aide: We Didn’t Even See Comey’s Let...
 1          1  ...  Ever get the feeling your life circles the rou...
 2          2  ...  Why the Truth Might Get You Fired October 29, ...
 3          3  ...  Videos 15 Civilians Killed In Single US Airstr...
 4          4  ...  Print \nAn Iranian woman has been sentenced to...
 ...      ...  ...                                                ...
 18280  20795  ...  Rapper T. I. unloaded on black celebrities who...
 18281  20796  ...  When the Green Bay Packers lost to the Washing...
 18282  20797  ...  The Macy’s of today grew from the union of sev...
 18283  20798  ...  NATO, Russia To Hold Parallel Exercises In Bal...
 18284  20799  ...    David Swanson is an author, activist, journa...
 
 [18285 rows x 5 columns],
 0        1
 1        0
 2        1
 3        1
 4        1
         ..
 18280    0
 18281    0
 18282    0
 18283    1
 18284    1

In [50]:
x.shape,y.shape

((18285, 5), (18285,))

In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [52]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
messages = x.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("a-zA-z"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)


In [56]:
vocab_size = 5000
onehot_repr = [one_hot(words,vocab_size) for words in corpus]

In [57]:
onehot_repr

[[4442, 1108, 2899, 2642, 1861, 256, 4620, 4303, 3552, 4804, 3945],
 [4621, 933, 1818, 945, 2918, 2408, 906],
 [3858, 4955, 589, 3292],
 [877, 4188, 2545, 3033, 1944, 1912, 3263],
 [3509, 2918, 4363, 4060, 2863, 4151, 2918, 1178, 2977, 2687],
 [699,
  1864,
  3292,
  2606,
  615,
  2242,
  3220,
  2302,
  3965,
  3853,
  1396,
  1588,
  1558,
  2242,
  906],
 [4552, 2849, 3724, 1059, 1799, 2571, 1423, 3354, 1384, 2388, 1191],
 [3114, 3196, 2579, 1220, 3524, 463, 2242, 491, 1384, 2388, 1191],
 [1160, 4611, 3115, 913, 2671, 2670, 2824, 1825, 4220, 3097],
 [996, 235, 1442, 1372, 4150, 1871, 1874, 4973],
 [4449, 3811, 4547, 4588, 2270, 3031, 4339, 3562, 973, 735, 3749],
 [1944, 3409, 256, 2670, 2242, 3524],
 [4283, 1800, 772, 1601, 3343, 4192, 2245, 1166, 4464, 3942],
 [2520, 1387, 1059, 1209, 4708, 4735, 235, 1384, 2388, 1191],
 [346, 682, 581, 570, 3709, 1384, 2388, 1191],
 [601, 1096, 4738, 246, 4395, 4428, 2047, 4255, 3868, 3813, 4304, 3663],
 [4543, 4080, 2439],
 [973, 2797, 4106, 391

applying the pad sequence : makes sentence length equal


In [58]:
sent_length = 20
embedd_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)
print(embedd_docs)

[[   0    0    0 ... 3552 4804 3945]
 [   0    0    0 ... 2918 2408  906]
 [   0    0    0 ... 4955  589 3292]
 ...
 [   0    0    0 ... 1384 2388 1191]
 [   0    0    0 ... 3129 2289 2058]
 [   0    0    0 ... 1094 2442 2081]]


In [60]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss ='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
import numpy as np
x_final = np.array(embedd_docs)
y_final = np.array(y)

In [63]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest =  train_test_split(x_final,y_final,test_size = 0.25,random_state = 42)


In [65]:
model.fit(Xtrain,ytrain,validation_data=(Xtest,ytest),epochs = 10,batch_size =64 )

Epoch 1/10
215/215 [==============================] - 13s 48ms/step - loss: 0.3219 - accuracy: 0.8436 - val_loss: 0.1952 - val_accuracy: 0.9134
Epoch 2/10
215/215 [==============================] - 9s 43ms/step - loss: 0.1303 - accuracy: 0.9503 - val_loss: 0.2023 - val_accuracy: 0.9167
Epoch 3/10
215/215 [==============================] - 8s 36ms/step - loss: 0.0892 - accuracy: 0.9669 - val_loss: 0.2395 - val_accuracy: 0.9178
Epoch 4/10
215/215 [==============================] - 9s 42ms/step - loss: 0.0525 - accuracy: 0.9824 - val_loss: 0.2518 - val_accuracy: 0.9127
Epoch 5/10
215/215 [==============================] - 9s 42ms/step - loss: 0.0324 - accuracy: 0.9901 - val_loss: 0.3107 - val_accuracy: 0.9086
Epoch 6/10
215/215 [==============================] - 10s 45ms/step - loss: 0.0215 - accuracy: 0.9932 - val_loss: 0.3348 - val_accuracy: 0.9119
Epoch 7/10
215/215 [==============================] - 12s 58ms/step - loss: 0.0106 - accuracy: 0.9968 - val_loss: 0.4931 - val_accuracy: 0.9

In [66]:
y_pred = model.predict(Xtest)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest,y_pred)

143/143 [==============================] - 1s 6ms/step


array([[2358,  242],
       [ 165, 1807]])

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,y_pred)

0.9109798775153106